<div style="
    background-color: #1e1e1e; 
    color: #e0e0e0; 
    font-family: 'Consolas', 'Monaco', 'Courier New', monospace; 
    padding: 25px; 
    border-radius: 8px; 
    border: 1px solid #333; 
    box-shadow: 0 4px 15px rgba(0,0,0,0.5);
    max-width: 950px;">

<h1 style="
        color: #4fc3f7; 
        border-bottom: 2px solid #0288d1; 
        padding-bottom: 10px; 
        margin-top: 0;
        font-family: 'Segoe UI', sans-serif;
        letter-spacing: 1px;">
        &lt;Optimization: Direct Hashing & Pruning (DHP) /&gt;
    </h1>

 <p style="font-size: 1.05em; color: #b0bec5;">
        The standard Apriori algorithm faces a critical bottleneck during the generation of 
        <strong style="color: #fff;">2-itemsets (C₂)</strong>. DHP addresses this by aggressively pruning candidate pairs early using a hash-based filtering technique.
    </p>

<div style="
        background-color: #263238; 
        border-left: 5px solid #ff5252; 
        padding: 15px; 
        margin: 25px 0; 
        border-radius: 4px;">
        <h3 style="margin-top: 0; color: #ff8a80; font-family: 'Segoe UI', sans-serif;">
            ⚠️ The Bottleneck: Combinatorial Explosion
        </h3>
        <p style="margin-bottom: 0; font-size: 0.95em;">
            If you have 1,000 frequent items in <code>L₁</code>, standard Apriori must generate 
            <code style="color: #ff5252; background-color: rgba(255,255,255,0.1); padding: 2px 6px; border-radius: 4px;">C(1000, 2) ≈ 500,000</code> 
            candidate pairs. Checking all these against the database is slow and memory-intensive.
        </p>
    </div>

<h2 style="color: #81d4fa; font-family: 'Segoe UI', sans-serif;">// How DHP Works</h2>
    <p>
        DHP <strong>"hacks" the first pass</strong>. While counting single items (k=1), it simultaneously gathers data about pairs.
    </p>

<div style="margin-left: 10px;">
        <div style="margin-bottom: 15px;">
            <strong style="color: #fff; font-size: 1.1em;">1. The Hash Function</strong><br>
            During the initial scan, every pair of items in a transaction is passed through a function:
            <div style="
                background-color: #000; 
                color: #a5d6a7; 
                padding: 10px; 
                border-radius: 4px; 
                margin: 10px 0; 
                border-left: 3px solid #66bb6a;">
                bucket_index = ( (order(x) * 10) + order(y) ) % N
            </div>
        </div>

 <div style="margin-bottom: 15px;">
            <strong style="color: #fff; font-size: 1.1em;">2. The Bucket Count</strong><br>
            A Hash Table tracks counts. If a pair maps to bucket #5, we increment bucket #5. We don't store <em>which</em> pair it was, just the frequency.
        </div>

<div>
            <strong style="color: #fff; font-size: 1.1em;">3. The Golden Rule (Pruning)</strong><br>
            After the scan, we check the buckets. <br>
            <span style="color: #ffcc80;">If a bucket's count < min_support, ALL pairs mapping to that bucket are discarded immediately.</span>
        </div>
    </div>

<div style="
        background-color: #1b5e20; 
        background: linear-gradient(145deg, #1b5e20 0%, #2e7d32 100%);
        padding: 20px; 
        margin: 30px 0; 
        border-radius: 6px; 
        color: #e8f5e9;">
        <h3 style="margin-top: 0; color: #fff; font-family: 'Segoe UI', sans-serif;">💡 Concrete Example</h3>
        <p style="margin: 5px 0;"><strong>Scenario:</strong> <code>min_support = 10</code></p>
        <ul style="list-style-type: square; padding-left: 20px;">
            <li>We analyze pair <code>{Milk, Bread}</code>.</li>
            <li>Hash function maps it to <strong>Bucket #5</strong>.</li>
            <li>Total count in Bucket #5 is found to be <strong>8</strong>.</li>
        </ul>
        <hr style="border: 0; border-top: 1px solid rgba(255,255,255,0.3); margin: 10px 0;">
        <p style="margin-bottom: 0; font-weight: bold;">
            Conclusion: Even if all 8 hits were {Milk, Bread}, 8 < 10. This pair is impossible. DROP IT.
        </p>
    </div>

<h2 style="color: #81d4fa; font-family: 'Segoe UI', sans-serif;">// Performance Comparison</h2>
    
 <table style="width: 100%; border-collapse: collapse; margin-top: 15px; font-size: 0.95em;">
        <thead>
            <tr style="border-bottom: 2px solid #4fc3f7;">
                <th style="text-align: left; padding: 12px; color: #4fc3f7;">Feature</th>
                <th style="text-align: left; padding: 12px; color: #b0bec5;">Standard Apriori</th>
                <th style="text-align: left; padding: 12px; color: #fff;">Apriori + DHP</th>
            </tr>
        </thead>
        <tbody>
            <tr style="border-bottom: 1px solid #424242;">
                <td style="padding: 12px; color: #e0e0e0;"><strong>Candidate Gen (C₂)</strong></td>
                <td style="padding: 12px; color: #9e9e9e;">Blindly joins all L₁ items ($\approx n^2/2$).</td>
                <td style="padding: 12px; color: #81c784;">Only generates pairs from frequent buckets.</td>
            </tr>
            <tr style="border-bottom: 1px solid #424242;">
                <td style="padding: 12px; color: #e0e0e0;"><strong>DB Size</strong></td>
                <td style="padding: 12px; color: #9e9e9e;">Full scan every time.</td>
                <td style="padding: 12px; color: #81c784;">Can "trim" DB by removing useless transactions.</td>
            </tr>
        </tbody>
    </table>

</div>
